In [ ]:
from __future__ import absolute_import, division, print_function

import tensorflow as tf

import tensorflow.keras.backend as K
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras import regularizers
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.regularizers import l2

from tensorflow import keras
from tensorflow.keras import models
from tensorflow.keras.applications.inception_v3 import preprocess_input

import cv2
import os
import random
import collections
from collections import defaultdict

from shutil import copy
from shutil import copytree, rmtree
import shutil

import numpy as np

import matplotlib.pyplot as plt
import matplotlib.image as img
%matplotlib inline

import h5py
import scipy.io as sio

In [ ]:
# Clone the entire repo.
!git clone -l -s https://github.com/mhmedwagih1324/FCRN.git model 
%cd model
!ls

Cloning into 'model'...
remote: Enumerating objects: 50, done.
remote: Counting objects: 100% (50/50), done.
remote: Compressing objects: 100% (39/39), done.
remote: Total 81 (delta 25), reused 27 (delta 11), pack-reused 31
Unpacking objects: 100% (81/81), done.
/content/model
fcrn.py		    README.md	    testIdxs.txt   training_results.xlsx
flow_transforms.py  result_521.png  test.py	   train.py
loader.py	    result_599.png  trainIdxs.txt  weights.py


In [ ]:
# Download data and extract it to folder
#!gdown https://drive.google.com/uc?id=132jSIsMoOBi0bgojLCqoe0J_YPKoKlN2 --output ready_dataset.mat
!gdown https://drive.google.com/uc?id=1LYiMQcuutDS8Fi8zfEBb9wOALcpT9C1B --output augmented_dataset.mat
!gdown https://drive.google.com/uc?id=1-6-CzjKV1Y8BWn2MhbYY_Uhu8JNH5wTS --output nyu_depth_v2_labeled.mat
!gdown https://drive.google.com/uc?id=1-5jdx-8Ri5zy3MbVzXs2mgj5nND_W0HD --output NYU_ResNet-UpProj.npy
!gdown https://drive.google.com/uc?id=1-2FUw0gzN85EsA8Ip1pPRwoE-VozPUw9 --output resnet50-19c8e357.pth
!gdown https://drive.google.com/uc?id=1rOzJuAPTodzc93SvyiX4iiwjOA4h9JUZ --output our_checkpoint.pth.tar

Downloading...
From: https://drive.google.com/uc?id=132jSIsMoOBi0bgojLCqoe0J_YPKoKlN2
To: /content/model/ready_dataset.mat
445MB [00:04, 103MB/s] 
Downloading...
From: https://drive.google.com/uc?id=1-6-CzjKV1Y8BWn2MhbYY_Uhu8JNH5wTS
To: /content/model/nyu_depth_v2_labeled.mat
2.97GB [00:58, 50.9MB/s]
Downloading...
From: https://drive.google.com/uc?id=1-5jdx-8Ri5zy3MbVzXs2mgj5nND_W0HD
To: /content/model/NYU_ResNet-UpProj.npy
255MB [00:01, 127MB/s]
Downloading...
From: https://drive.google.com/uc?id=1-2FUw0gzN85EsA8Ip1pPRwoE-VozPUw9
To: /content/model/resnet50-19c8e357.pth
103MB [00:00, 117MB/s]  
Downloading...
From: https://drive.google.com/uc?id=1rOzJuAPTodzc93SvyiX4iiwjOA4h9JUZ
To: /content/model/our_checkpoint.pth.tar
763MB [00:02, 261MB/s]


In [ ]:
# skip this if the pretrained weights file is our_checkpoint.pth.tar
# change the block that looks the same in line 77 in train.py
#-----------------------------------------------------------
    model.load_state_dict(load_weights(model, weights_file, dtype))
    #chkp = torch.load('checkpoint.pth.tar')
    #model.load_state_dict(chkp['state_dict'])
#-----------------------------------------------------------

In [ ]:
!python train.py

Loading data......
/content/model/loader.py:16: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  self.nyu = h5py.File(self.data_path)
Loading model......
resnet50 loaded.
resnet50_pretrained_dict loaded.
model_dict updated.
model_dict loaded.
loss_fn set.
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:3121: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))
val_error before train: 0.49126771092414856
optimizer set.
Starting train epoch 1

In [ ]:
%cp /content/model/checkpoint.pth.tar '/content/drive/My Drive/Grad Project/'

In [ ]:
!python test.py

#Trash

In [ ]:
import scipy.io as sio
import numpy as np

def showImage(mat_fname):
  """
  adds an image to a mat file

  Args:
    - mat_fname(string): path to the mat file
    - image_arr(ndarray): the new image of shape:(height, width, 3)
    - depth_arr(ndarray): the depth of the new image of shape:(height, width)
  """
  # load the mat file into a dictionary then into two ndarrays
  mat_dict = sio.loadmat(mat_fname)
  imgs = mat_dict['images']
  dpts = mat_dict['depths']

  image1 = imgs

  # compine the arrays into a dictionary and overwrite the dictionary on the mat file
  mat_dict = {'images':imgs, 'depths':dpts}
  sio.savemat(mat_fname, mat_dict, do_compression=True)



In [ ]:
def get_data_extract(link, fname, folder):
  tf.keras.utils.get_file(
  fname,
  link,
  cache_subdir=folder,
  extract=True,
  archive_format='tar',
  cache_dir=None
  )
  print("Dataset downloaded and extracted!")

In [ ]:
#get_data_extract('http://horatio.cs.nyu.edu/mit/silberman/nyu_depth_v2/nyu_depth_v2_labeled.mat', 'nyu_depth_v2_labeled.mat', '/content/model/')
#get_data_extract('http://campar.in.tum.de/files/rupprecht/depthpred/NYU_ResNet-UpProj.npy', 'NYU_ResNet-UpProj.npy', '/content/model/')
#get_data_extract('https://download.pytorch.org/models/resnet50-19c8e357.pth', 'resnet50-19c8e357.pth', '/content/')

In [ ]:
# skip this
# checking the dimensions of dataset
nyu = h5py.File('nyu_depth_v2_labeled.mat', 'r')
imgs = nyu['images']
dpts = nyu['depths']
print(imgs.shape)
print(dpts.shape)
img = imgs[5].transpose(2, 1, 0)
dpt = dpts[5].transpose(1, 0)
print(img.shape)
print(dpt.shape)


(1449, 3, 640, 480)
(1449, 640, 480)
(480, 640, 3)
(480, 640)


In [ ]:
# skip this
nyu = h5py.File('/content/drive/My Drive/Dataset/myMat (1).mat', 'r')
imgs = nyu['imgs']
dpts = nyu['dpts']
print(imgs.shape)
print(dpts.shape)
img = imgs[0].transpose(2, 1, 0)
print(img.shape)

(2, 3, 640, 480)
(2, 640, 480)
(480, 640, 3)


In [ ]:
# skip this
#change in train.py
'''
line 72 ==> change the path to the .pth file in the main function to be '/content/resnet50-19c8e357.pth'
'''
'''
line 187 ==>  print('loss:', loss.data.cpu()[0])  change to ===> print('loss:', loss.data.cpu())
'''
'''
line 50 ==> data_path = 'nyu_depth_v2_labeled.mat' change to ==> data_path = 'ready_dataset.mat'
'''
#change in weights.py
'''
Line 8 => add an argument in the line of 'data_dict = np.load' in the load_weights function 
so that the whole line is the following line:
data_dict = np.load(weights_file, encoding='latin1', allow_pickle=True).item()
'''

# upload trainIdxs.txt and testIdxs.txt from local machine



"\nadd an argument in the line of 'data_dict = np.load' in the load_weights function \nso that the whole line is the following line:\ndata_dict = np.load(weights_file, encoding='latin1', allow_pickle=True).item()\n"

In [ ]:
https://drive.google.com/file/d/1-6-CzjKV1Y8BWn2MhbYY_Uhu8JNH5wTS/view?usp=sharing
https://drive.google.com/file/d/1-5jdx-8Ri5zy3MbVzXs2mgj5nND_W0HD/view?usp=sharing
https://drive.google.com/file/d/1-2FUw0gzN85EsA8Ip1pPRwoE-VozPUw9/view?usp=sharing

102457344/254553442 [===========>..................] - ETA: 37s

In [ ]:
from google.colab import files
for i in range(51):
  files.download('gt_depth_epoch_{}.png'.format(i))
  files.download('input_rgb_epoch_{}.png'.format(i))
  files.download('pred_depth_epoch_{}.png'.format(i))
  if i == 0 or i>36:
    continue
  if i<10:
    files.download('Test_pred_depth_0000{}.png'.format(i))
    files.download('Test_gt_depth_0000{}.png'.format(i))
    continue
  files.download('Test_pred_depth_000{}.png'.format(i))
  files.download('Test_gt_depth_000{}.png'.format(i))

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>